In [ ]:
from datetime import date 
today = date.today()
print("Today's date:", today)

In [ ]:
import datetime
print("Started at "+str(datetime.datetime.now()))

from neo4j import GraphDatabase

URI = "neo4j://20.98.50.241:7687"
AUTH = ("neo4j", "P@55w0rd@2023")

def create_customer(tx, customerID,companyName,contactName,contactTitle):
    tx.run ("MERGE (c:Customer {customerID:$customerID}) SET c.companyName=$companyName,c.contactName=$contactName,c.contactTitle=$contactTitle",customerID=customerID,companyName=companyName,contactName=contactName,contactTitle=contactTitle)
def create_supplier(tx, supplierID):
    tx.run ("MERGE (s:Supplier {supplierID:$supplierID})",supplierID=supplierID)
def create_order(tx, orderID):
    tx.run ("MERGE (o:Order {orderID:$orderID})",orderID=orderID)
def create_product(tx, productID,productName,unitPrice,quantityPerUnit,unitsInStock,unitsOnOrder,reorderLevel,discontinued):
    tx.run ("MERGE (p:Product {productID:$productID}) SET p.productName=$productName,p.quantityPerUnit=toInteger($quantityPerUnit),p.unitPrice = toFloat($unitPrice), p.unitsInStock = toInteger($unitsInStock), p.unitsOnOrder = toInteger($unitsOnOrder), p.reorderLevel = toInteger($reorderLevel), p.discontinued = $discontinued",productID=productID,productName=productName,quantityPerUnit=quantityPerUnit,unitPrice=unitPrice,unitsInStock=unitsInStock,unitsOnOrder=unitsOnOrder,reorderLevel=reorderLevel,discontinued=discontinued)
def create_category(tx, categoryID, categoryName, description):
    tx.run ("MERGE (cat:Category {categoryID:$categoryID,categoryName:$categoryName,description:$description})",categoryID=categoryID,categoryName=categoryName,description=description)

customers_df = spark.read.format("csv").option("header","true").load("Files/Northwind/customers.csv")
customer_df=customers_df.toPandas()
print("Loaded customers");
print(datetime.datetime.now())

customers_df.write
    .format("org.neo4j.spark.DataSource")
    .option("url", "bolt://20.98.50.241:7687")
    .option("relationship", "PLAYS")
    .option("relationship.save.strategy", "keys")
    .option("relationship.source.labels", ":Musician")
    .option("relationship.source.save.mode", "overwrite")
    .option("relationship.source.node.keys", "name:name")
    .option("relationship.target.labels", ":Instrument")
    .option("relationship.target.node.keys", "instrument:name")
    .option("relationship.target.save.mode", "overwrite")
    .save()


suppliers_df = spark.read.format("csv").option("header","true").load("Files/Northwind/suppliers.csv")
supplier_df=suppliers_df.toPandas()
print("Loaded Suppliers");
print(datetime.datetime.now())
categories_df = spark.read.format("csv").option("header","true").load("Files/Northwind/categories.csv")
category_df=categories_df.toPandas()
print("Loaded Categories");
print(datetime.datetime.now())
products_df = spark.read.format("csv").option("header","true").load("Files/Northwind/products.csv")
product_df=products_df.toPandas()
print("Loaded Products");
print(datetime.datetime.now())
orders_df = spark.read.format("csv").option("header","true").load("Files/Northwind/orders.csv")
order_df=orders_df.toPandas()
print("Loaded Orders");
print(datetime.datetime.now())

with GraphDatabase.driver(URI, auth=AUTH) as driver:
    with driver.session() as session:
        for index, customer in customer_df.iterrows():
           session.execute_write(create_customer, customer[0],customer[1],customer[2],customer[3])
        print("after customer loop"+str(datetime.datetime.now())   
        for index, supplier in supplier_df.iterrows():
           session.execute_write(create_supplier, supplier[0])
        print("after supplier loop"+str(datetime.datetime.now())    
        for index, order in order_df.iterrows():
           session.execute_write(create_order, order[0])
        print("after orders loop"+str(datetime.datetime.now())    
        for index, category in category_df.iterrows():
           session.execute_write(create_category, category[0],category[1],category[2])
        print("after catergory loop"+str(datetime.datetime.now())    
        for index, product in product_df.iterrows():
           session.execute_write(create_product, product[0],product[1],product[4],product[5],product[6],product[7],product[8],product[9])
print("Completed at:"+str(datetime.datetime.now()))